In [1]:
import numpy as np
import os
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
IMAGE_SIZE = 150
LABELS = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
NUM_CLASSES = len(LABELS)
# *** REPLACE with the path from your Google Drive ***
DATASET_PATH = '/content/drive/MyDrive/Tumor_Detection'

In [3]:
print("--- Starting Data Loading ---")
X = [] # To hold image data
Y = [] # To hold labels

def load_images_from_folder(base_path, folder_name):
    full_path = os.path.join(base_path, folder_name)
    if not os.path.exists(full_path):
        print(f"Path not found: {full_path}. Please check your DATASET_PATH.")
        return
    for label in LABELS:
        folderPath = os.path.join(full_path, label)
        print(f"Loading images from: {folderPath}")
        for j in os.listdir(folderPath):
            img_path = os.path.join(folderPath, j)
            try:
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
                    X.append(img)
                    Y.append(label)
            except Exception as e:
                print(f"Could not load image {img_path}: {e}")

load_images_from_folder(DATASET_PATH, 'Training')
load_images_from_folder(DATASET_PATH, 'Testing')

print(f"Total Images Loaded: {len(X)}")

if len(X) == 0:
    print("No images loaded. Exiting script.")
    exit()

X = np.array(X)
Y = np.array(Y)
X, Y = shuffle(X, Y, random_state=101)

# Split into Training (80%), Validation (10%), and Test (10%)
X_train_raw, X_test, y_train_raw, y_test_raw = train_test_split(X, Y, test_size=0.1, random_state=101)
X_train, X_val, y_train_raw, y_val_raw = train_test_split(
    X_train_raw, y_train_raw, test_size=0.1/0.9, random_state=101
)

def encode_labels(raw_labels):
    encoded = [LABELS.index(i) for i in raw_labels]
    return tf.keras.utils.to_categorical(encoded, num_classes=NUM_CLASSES)

y_train = encode_labels(y_train_raw)
y_val = encode_labels(y_val_raw)
y_test = encode_labels(y_test_raw)

--- Starting Data Loading ---
Loading images from: /content/drive/MyDrive/Tumor_Detection/Training/glioma_tumor
Loading images from: /content/drive/MyDrive/Tumor_Detection/Training/meningioma_tumor
Loading images from: /content/drive/MyDrive/Tumor_Detection/Training/no_tumor
Loading images from: /content/drive/MyDrive/Tumor_Detection/Training/pituitary_tumor
Loading images from: /content/drive/MyDrive/Tumor_Detection/Testing/glioma_tumor
Loading images from: /content/drive/MyDrive/Tumor_Detection/Testing/meningioma_tumor
Loading images from: /content/drive/MyDrive/Tumor_Detection/Testing/no_tumor
Loading images from: /content/drive/MyDrive/Tumor_Detection/Testing/pituitary_tumor
Total Images Loaded: 3266


In [4]:
print("\n--- Building and Fine-Tuning VGG16 Model ---")

# Load VGG16 without the top layers
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
)

# Freeze the majority of the VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last 4 convolutional layers for fine-tuning
# This allows the model to learn medical-specific features
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Construct the full model
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model (Use a very low learning rate for fine-tuning)
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), # Lower LR than before!
    metrics=['accuracy']
)

model.summary()


--- Building and Fine-Tuning VGG16 Model ---
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,911,556 (72.14 MB)

 Trainable params: 11,276,292 (43.02 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [5]:
print("\n--- Setting up Data Generators ---")

# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only normalization for validation and test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_test_datagen.flow(X_val, y_val, batch_size=32)
test_datagen = val_test_datagen # Use the same normalization for test set


--- Setting up Data Generators ---


In [6]:
print("\n--- Starting Model Training ---")

MODEL_SAVE_PATH = 'brain_tumor_detection_model.keras'
callbacks = [
    ModelCheckpoint(
        filepath=MODEL_SAVE_PATH,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=7, # Increased patience since we are training more robustly
        restore_best_weights=True
    )
]

# Train the model using the generators
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32, # Calculate steps based on batch size
    epochs=50,                          # Increased epochs for better learning
    validation_data=val_generator,
    validation_steps=len(X_val) // 32,
    callbacks=callbacks,
    verbose=1
)

# Load and evaluate the best model
best_model = tf.keras.models.load_model(MODEL_SAVE_PATH)

# Use the test generator for evaluation (only normalization is applied)
loss, accuracy = best_model.evaluate(test_datagen.flow(X_test, y_test, batch_size=32), verbose=0)
print(f"\n✅ Final Model Test Accuracy: {accuracy*100:.2f}%")

print(f"\nModel saved as '{MODEL_SAVE_PATH}' for UI deployment.")


--- Starting Model Training ---


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.3393 - loss: 1.4403
Epoch 1: val_accuracy improved from -inf to 0.69063, saving model to brain_tumor_detection_model.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 43s 374ms/step - accuracy: 0.3405 - loss: 1.4382 - val_accuracy: 0.6906 - val_loss: 0.8437
Epoch 2/50
 1/81 ━━━━━━━━━━━━━━━━━━━━ 8s 104ms/step - accuracy: 0.5312 - loss: 1.1047

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy improved from 0.69063 to 0.69375, saving model to brain_tumor_detection_model.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5312 - loss: 1.1047 - val_accuracy: 0.6938 - val_loss: 0.8309
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.6105 - loss: 0.9307
Epoch 3: val_accuracy improved from 0.69375 to 0.76562, saving model to brain_tumor_detection_model.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 20s 248ms/step - accuracy: 0.6110 - loss: 0.9299 - val_accuracy: 0.7656 - val_loss: 0.6586
Epoch 4/50
 1/81 ━━━━━━━━━━━━━━━━━━━━ 8s 104ms/step - accuracy: 0.7812 - loss: 0.6147
Epoch 4: val_accuracy did not improve from 0.76562
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7812 - loss: 0.6147 - val_accuracy: 0.7656 - val_loss: 0.6555
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.7174 - loss: 0.7129
Epoch 5: val_accuracy improved from 0.76562 to 0.81875, saving model to brain_tumor_detection_model.keras
81/81 ━━━━━━━━━━━━━━━━